In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import scipy as sp

from ad_afqmc import config

config.afqmc_config["use_mpi"] = False
config.setup_jax()
comm = config.setup_comm()

from jax import vmap, jit, numpy as jnp, random, lax, jvp, scipy as jsp
import matplotlib.pyplot as plt
import pickle
from functools import reduce


from ad_afqmc import (
    driver,
    pyscf_interface,
    launch_script,
    linalg_utils,
    sampling,
    lattices,
    propagation,
    wavefunctions,
    hamiltonian,
)

from pyscf import fci, gto, scf, mp, ao2mo

import itertools
from functools import partial

np.set_printoptions(precision=5, suppress=True)

In [ ]:
@jit
def ham_element_hubbard(x, y, u, t_matrix):
    # coulomb
    diff = jnp.array((jnp.bitwise_xor(x[0], y[0]), jnp.bitwise_xor(x[1], y[1])))
    diff_count = jnp.array((jnp.sum(diff[0]), jnp.sum(diff[1])))
    onsite_1e_energy = jnp.dot(x[0] + x[1], jnp.diag(t_matrix))
    on_site_coulomb = u * jnp.bitwise_and(x[0], x[1]).sum()
    on_site = (jnp.sum(diff_count) == 0) * (onsite_1e_energy + on_site_coulomb)

    # hopping
    is_hopping = jnp.sum(diff_count) == 2
    diff_pos = jnp.nonzero(diff, size=2)
    spin_idx = diff_pos[0][0]
    site_1 = diff_pos[1][0]
    site_2 = diff_pos[1][1]
    t_element = t_matrix[site_1, site_2]
    # parity
    min_site = jnp.minimum(site_1, site_2)
    max_site = jnp.maximum(site_1, site_2)
    sites = jnp.arange(len(x[0]))
    mask = (sites > min_site) & (sites < max_site)
    electrons_between = jnp.sum(y[spin_idx] * mask)
    parity_factor = jnp.power(-1.0, electrons_between)
    hopping = is_hopping * t_element * parity_factor

    return on_site + hopping


def make_basis_spin(n_sites, n_elec):
    # generate permutations using lexicographic order
    basis = []
    elec = np.zeros(n_sites)
    for i in range(n_elec):
        elec[-i - 1] = 1
    basis.append(elec.copy())
    # find next permutation
    while True:
        k = -1
        for i in range(n_sites - 1):
            if elec[i] < elec[i + 1]:
                k = i
        if k == -1:
            break
        l = k
        for i in range(k + 1, n_sites):
            if elec[k] < elec[i]:
                l = i
        elec[k], elec[l] = elec[l], elec[k]
        elec[k + 1 :] = np.flip(elec[k + 1 :])
        basis.append(elec.copy())
    return np.array(basis, dtype=int)


def make_basis(n_sites, n_elec: tuple):
    basis_up = make_basis_spin(n_sites, n_elec[0])
    if n_elec[0] == n_elec[1]:
        basis_down = basis_up
    else:
        basis_down = make_basis_spin(n_sites, n_elec[1])
    basis = itertools.product(basis_up, basis_down)
    return np.array(list(basis))


@jit
def calc_s2(sd_up, sd_dn):
    green_up = (
        sd_up.dot(jnp.linalg.inv(sd_up.T.conj().dot(sd_up))).dot(sd_up.T.conj())
    ).T
    green_dn = (
        sd_dn.dot(jnp.linalg.inv(sd_dn.T.conj().dot(sd_dn))).dot(sd_dn.T.conj())
    ).T
    spsm = sd_up.shape[1] - jnp.trace(green_up.dot(green_dn))
    sz = (sd_up.shape[1] - sd_dn.shape[1]) / 2
    return spsm + sz * (sz - 1)


@jit
def calc_green(sd):
    return (sd.dot(jnp.linalg.inv(sd.T.conj().dot(sd))).dot(sd.T.conj())).T


@jit
def calc_norm(sd):
    return jnp.linalg.det(sd.T.conj().dot(sd))


@jit
def normalize(walker):
    walker = vmap(lambda x: x / jnp.linalg.norm(x), in_axes=1, out_axes=1)(walker)
    return walker

In [ ]:
n_sites = 4
n_elec = (2, 2)
u = 4.0
lattice = lattices.one_dimensional_chain(n_sites)
t_matrix = -1 * lattice.create_adjacency_matrix()
ci_basis = jnp.array(make_basis(n_sites, n_elec), dtype=jnp.int32)
print(f"built basis, length: {len(ci_basis)}")
ham_element = partial(ham_element_hubbard, u=u, t_matrix=t_matrix)
ham_mat = vmap(vmap(ham_element, (None, 0)), (0, None))(ci_basis, ci_basis)
print("built hamiltonian")
ene_mat, vec = jnp.linalg.eigh(ham_mat)
print(ene_mat[:10])

## 2D Hubbard CPMC


### Square lattice (UHF trial)


In [ ]:
lattice = lattices.two_dimensional_grid(4, 4)
# lattice = lattices.one_dimensional_chain(6)
n_sites = lattice.n_sites
u = 4.0
n_elec = (8, 8)
# n_elec = (3, 3)

np.random.seed(0)

# make dummy molecule
mol = gto.Mole()
mol.nelectron = sum(n_elec)
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.verbose = 3
mol.build()

np.random.seed(0)
h1 = -1.0 * lattice.create_adjacency_matrix()
umf = scf.UHF(mol)
umf.get_hcore = lambda *args: h1
umf.get_ovlp = lambda *args: np.eye(n_sites)
# umf._eri = ao2mo.restore(8, integrals["h2"], n_sites)
umf._eri = None
get_vj = lambda mol, dm, **kw: np.array(
    [
        u * np.diag(np.diag(dm[0])),
        u * np.diag(np.diag(dm[1])),
    ]
)
get_vk = lambda mol, dm, **kw: np.array([np.zeros_like(dm[0]), np.zeros_like(dm[1])])


def get_jk(mol, dm, hermi=1, with_j=True, with_k=True, omega=None):
    vj = (
        get_vj(mol, dm)
        if with_j
        else np.array([np.zeros_like(dm[0]), np.zeros_like(dm[1])])
    )
    vk = (
        get_vj(mol, dm)
        if with_k
        else np.array([np.zeros_like(dm[0]), np.zeros_like(dm[1])])
    )
    return vj, vk


umf.get_jk = get_jk
dm_init = 1.0 * umf.init_guess_by_1e()
dm_init += 1.0 * np.random.randn(*dm_init.shape)
umf.kernel(dm_init)
mo1 = umf.stability()[0]
umf = umf.newton().run(mo1, umf.mo_occ)
mo1 = umf.stability()[0]
umf = umf.newton().run(mo1, umf.mo_occ)
umf.stability()

# raise

# rotate uhf to x axis to get ghf
gmf_coeff = np.zeros((2 * n_sites, n_elec[0] + n_elec[1]))
theta = np.pi / 4
gmf_coeff[:n_sites, : n_elec[0]] = np.cos(theta) * umf.mo_coeff[0][:, : n_elec[0]]
gmf_coeff[n_sites:, : n_elec[0]] = np.sin(theta) * umf.mo_coeff[0][:, : n_elec[0]]
gmf_coeff[:n_sites, n_elec[0] :] = np.sin(-theta) * umf.mo_coeff[1][:, : n_elec[1]]
gmf_coeff[n_sites:, n_elec[0] :] = np.cos(-theta) * umf.mo_coeff[1][:, : n_elec[1]]

# fci
if sum(n_elec) < 8:
    integrals = {}
    integrals["h0"] = 0.0
    integrals["h1"] = h1
    h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
    for i in range(n_sites):
        h2[i, i, i, i] = u
    integrals["h2"] = ao2mo.restore(8, h2, n_sites)

    ci = fci.FCI(mol)
    e, ci_coeffs = ci.kernel(
        h1e=integrals["h1"], eri=integrals["h2"], norb=n_sites, nelec=n_elec
    )
    print(f"fci energy: {e}")

# ad afqmc
# pyscf_interface.prep_afqmc(umf, basis_coeff=np.eye(n_sites), integrals=integrals)
options = {
    "dt": 0.005,
    "n_eql": 5,
    "n_ene_blocks": 1,
    "n_sr_blocks": 5,
    "n_blocks": 100,
    "n_prop_steps": 50,
    "n_walkers": 50,
    "seed": 98,
    "walker_type": "uhf",
    "trial": "uhf",
    "save_walkers": False,
}

# ham_data, ham, prop, trial, wave_data, sampler, observable, options = (
#     launch_script.setup_afqmc(options)
# )
options = launch_script.read_options(options)
ham_data = {}
ham_data["h1"] = jnp.array([integrals["h1"], integrals["h1"]])
ham_data["u"] = u
ham = hamiltonian.hamiltonian(n_sites)
prop = propagation.propagator_cpmc(
    dt=options["dt"],
    n_walkers=options["n_walkers"],
)
trial = wavefunctions.ghf_cpmc(n_sites, n_elec)

# wave_data["mo_coeff"] = [
#     umf.mo_coeff[0][:, : n_elec[0]],
#     umf.mo_coeff[1][:, : n_elec[1]],
# ]
wave_data = {}
wave_data["mo_coeff"] = gmf_coeff[:, : n_elec[0] + n_elec[1]]
sampler = sampling.sampler_cpmc(
    n_ene_blocks=options["n_ene_blocks"],
    n_sr_blocks=options["n_sr_blocks"],
    n_blocks=options["n_blocks"],
)

e_afqmc, err_afqmc = driver.afqmc(
    ham_data, ham, prop, trial, wave_data, sampler, None, options, comm
)

### Nearest neighbor interactions


In [ ]:
lattice = lattices.two_dimensional_grid(4, 4)
n_sites = lattice.n_sites
u = 4.0
u_1 = 0.0
n_elec = (8, 8)

np.random.seed(0)

integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
neighbors = set()
for i in range(n_sites):
    h2[i, i, i, i] = u
    site_i = lattice.sites[i]
    for site_j in lattice.get_nearest_neighbors(site_i):
        j = int(lattice.get_site_num(site_j))
        h2[i, i, j, j] = u_1
        h2[j, j, i, i] = u_1
        if i < j:
            neighbors.add((i, j))
neighbors = tuple(neighbors)
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

# make dummy molecule
mol = gto.Mole()
mol.nelectron = sum(n_elec)
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.build()

mf = scf.RHF(mol)
mf.get_hcore = lambda *args: integrals["h1"]
mf.get_ovlp = lambda *args: np.eye(n_sites)
mf._eri = ao2mo.restore(8, integrals["h2"], n_sites)
mf.kernel()

np.random.seed(0)

umf = scf.UHF(mol)
umf.get_hcore = lambda *args: integrals["h1"]
umf.get_ovlp = lambda *args: np.eye(n_sites)
umf._eri = ao2mo.restore(8, integrals["h2"], n_sites)
dm_init = 1.0 * umf.init_guess_by_1e()
# for i in range(n_sites // 2):
#     dm_init[0, 2 * i, 2 * i] = 1.0
#     dm_init[1, 2 * i + 1, 2 * i + 1] = 1.0
dm_init += 1.0 * np.random.randn(*dm_init.shape)
umf.kernel(dm_init)
mo1 = umf.stability()[0]
umf = umf.newton().run(mo1, umf.mo_occ)
mo1 = umf.stability()[0]
umf = umf.newton().run(mo1, umf.mo_occ)
umf.stability()

# rotate uhf to x axis to get ghf
gmf_coeff = np.zeros((2 * n_sites, n_elec[0] + n_elec[1]))
gmf_coeff[:n_sites, : n_elec[0]] = umf.mo_coeff[0][:, : n_elec[0]] / 2**0.5
gmf_coeff[n_sites:, : n_elec[0]] = umf.mo_coeff[0][:, : n_elec[0]] / 2**0.5
gmf_coeff[:n_sites, n_elec[0] :] = umf.mo_coeff[1][:, : n_elec[1]] / 2**0.5
gmf_coeff[n_sites:, n_elec[0] :] = -umf.mo_coeff[1][:, : n_elec[1]] / 2**0.5

# fci
if sum(n_elec) < 8:
    ci = fci.FCI(mol)
    e, ci_coeffs = ci.kernel(
        h1e=integrals["h1"], eri=integrals["h2"], norb=n_sites, nelec=n_elec
    )
    print(f"fci energy: {e}")

# ad afqmc
pyscf_interface.prep_afqmc(umf, basis_coeff=np.eye(n_sites), integrals=integrals)
options = {
    "dt": 0.005,
    "n_eql": 5,
    "n_ene_blocks": 1,
    "n_sr_blocks": 5,
    "n_blocks": 100,
    "n_prop_steps": 50,
    "n_walkers": 50,
    "seed": 98,
    "walker_type": "uhf",
    "save_walkers": True,
}

ham_data, ham, prop, trial, wave_data, sampler, observable, options = (
    launch_script.setup_afqmc(options)
)


# comment out this block for doing ph-afqmc
prop = propagation.propagator_cpmc_nn(
    dt=options["dt"],
    n_walkers=options["n_walkers"],
    neighbors=neighbors,
)
trial = wavefunctions.ghf_cpmc(n_sites, n_elec)
wave_data["mo_coeff"] = gmf_coeff[:, : n_elec[0] + n_elec[1]]
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)
ham_data["u"] = u
ham_data["u_1"] = u_1

e_afqmc, err_afqmc = driver.afqmc(
    ham_data, ham, prop, trial, wave_data, sampler, observable, options, comm
)

### Triangular lattice (GHF trial)


In [ ]:
lattice = lattices.triangular_grid(6, 6, open_x=True)
n_sites = lattice.n_sites
u = 6.0
n_elec = (18, 18)

integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
for i in range(n_sites):
    h2[i, i, i, i] = u
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

# make dummy molecule
mol = gto.Mole()
mol.nelectron = sum(n_elec)
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.build()

mf = scf.RHF(mol)
mf.get_hcore = lambda *args: integrals["h1"]
mf.get_ovlp = lambda *args: np.eye(n_sites)
mf._eri = ao2mo.restore(8, integrals["h2"], n_sites)
mf.kernel()

umf = scf.UHF(mol)
umf.get_hcore = lambda *args: integrals["h1"]
umf.get_ovlp = lambda *args: np.eye(n_sites)
umf._eri = ao2mo.restore(8, integrals["h2"], n_sites)
dm_init = 1.0 * umf.init_guess_by_1e()
# for i in range(n_sites // 2):
#     dm_init[0, 2 * i, 2 * i] = 1.0
#     dm_init[1, 2 * i + 1, 2 * i + 1] = 1.0
dm_init += 1.0 * np.random.randn(*dm_init.shape)
umf.kernel(dm_init)
mo1 = umf.stability()[0]
umf = umf.newton().run(mo1, umf.mo_occ)
mo1 = umf.stability()[0]
umf = umf.newton().run(mo1, umf.mo_occ)
umf.stability()

# ghf
gmf = scf.GHF(mol)
gmf.get_hcore = lambda *args: sp.linalg.block_diag(integrals["h1"], integrals["h1"])
gmf.get_ovlp = lambda *args: np.eye(2 * n_sites)
gmf._eri = ao2mo.restore(8, integrals["h2"], n_sites)
dm_init = sp.linalg.block_diag(dm_init[0], dm_init[1])
dm_init += 2.0 * np.random.randn(*dm_init.shape)
gmf.kernel(dm_init)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)

# fci
if sum(n_elec) < 8:
    ci = fci.FCI(mol)
    e, ci_coeffs = ci.kernel(
        h1e=integrals["h1"], eri=integrals["h2"], norb=n_sites, nelec=n_elec
    )
    print(f"fci energy: {e}")

# ad afqmc
pyscf_interface.prep_afqmc(umf, basis_coeff=np.eye(n_sites), integrals=integrals)
options = {
    "dt": 0.005,
    "n_eql": 5,
    "n_ene_blocks": 1,
    "n_sr_blocks": 5,
    "n_blocks": 100,
    "n_prop_steps": 50,
    "n_walkers": 50,
    "seed": 98,
    "walker_type": "uhf",
    # "trial": "uhf",
    "save_walkers": False,
}

ham_data, ham, prop, trial, wave_data, sampler, observable, options = (
    launch_script.setup_afqmc(options)
)


# trial = wavefunctions.ghf(n_sites, n_elec)
prop = propagation.propagator_cpmc(dt=options["dt"], n_walkers=options["n_walkers"])
trial = wavefunctions.ghf_cpmc(n_sites, n_elec)
wave_data["mo_coeff"] = gmf.mo_coeff[:, : n_elec[0] + n_elec[1]]
rdm1 = gmf.make_rdm1()
wave_data["rdm1"] = jnp.array([rdm1[:n_sites, :n_sites], rdm1[n_sites:, n_sites:]])
gamma = np.arccosh(np.exp(prop.dt * u / 2))
const = jnp.exp(-prop.dt * u / 2)
ham_data["hs_constant"] = const * jnp.array(
    [[jnp.exp(gamma), jnp.exp(-gamma)], [jnp.exp(-gamma), jnp.exp(gamma)]]
)
ham_data["u"] = u

e_afqmc, err_afqmc = driver.afqmc(
    ham_data, ham, prop, trial, wave_data, sampler, observable, options, comm
)